<a href="https://colab.research.google.com/github/chiawld/UCL-Summer-School-Intro-to-AI-Course/blob/main/Day%201%20-%20Intro/Day1-Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import requests
import torch
from PIL import Image
from io import BytesIO
from sklearn.metrics.pairwise import cosine_similarity
from google.colab.patches import cv2_imshow
from transformers import pipeline
from transformers import BertModel, BertTokenizer

## Section 1: Language Processing
Application: Sentiment Analysis

In [2]:
# Load a pre-trained sentiment analysis pipeline from Hugging Face
sentiment_analysis = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
# Example text data
texts = [
    "I love sunny days!",
    "I hate when it rains all day.",
    "The weather is just perfect.",
    "I'm feeling quite sad today."
]

In [ ]:
# Perform sentiment analysis on each text
results = sentiment_analysis(texts)

In [ ]:
# Display the texts and their sentiment results
for text, result in zip(texts, results):
    print(f"Text: {text}\nSentiment: {result['label']} (Score: {result['score']})\n")

### Task 1: Test the limit of the sentiment model by changing the text input. Can you find a sentence that you think should be positive, but comes out as negative (or vice versa)? Do you agree with the scores, where 1 is highest? In this example, the fourth sentence is given a higher (negative) score than the second.

## Section 2: Computer Vision
Application: Object Detection

In [ ]:
# Function to download and display an image
def display_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert("RGB")
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    return img, np.array(img)

In [ ]:
# Load a pre-trained object detection model from TensorFlow Hub
detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

In [ ]:
# Example image URL
image_url = "https://images.unsplash.com/photo-1593642532973-d31b6557fa68"

In [ ]:
# Display the image
img, imgarray = display_image(image_url)

In [ ]:
# Convert the image to a tensor
img = img.resize((640, 480))  # Resize to match model's expected input size
img_tensor = tf.convert_to_tensor(np.array(img), dtype=tf.uint8)
img_tensor = tf.expand_dims(img_tensor, axis=0)  # Add batch dimension

In [ ]:
# Run the object detection model
result = detector(img_tensor)

In [ ]:
# Function to draw bounding boxes on the image
def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.5):
    colors = list(plt.cm.tab20(np.linspace(0, 1, 20)))
    font = cv2.FONT_HERSHEY_SIMPLEX
    for i in range(min(max_boxes, boxes.shape[0])):
        if scores[i] >= min_score:
            color = colors[int(class_names[i]) % len(colors)]
            ymin, xmin, ymax, xmax = tuple(boxes[i])
            image = cv2.rectangle(image, (int(xmin * img.width), int(ymin * img.height)),
                                  (int(xmax * img.width), int(ymax * img.height)), color, 2)
            image = cv2.putText(image, str(int(class_names[i])), (int(xmin * img.width), int(ymin * img.height) - 10),
                                font, 0.5, color, 2)
    return image

In [ ]:
# Extract detection results
boxes = result["detection_boxes"][0].numpy()
class_names = result["detection_classes"][0].numpy()
scores = result["detection_scores"][0].numpy()

In [ ]:
# Draw bounding boxes on the image
output_img = draw_boxes(np.array(img), boxes, class_names, scores)

In [ ]:
# Display the image with bounding boxes
cv2_imshow(output_img)

### Task 2: Test the limit of the model. Can you find a complex image that it might struggle with?

## Section 3: Computer Vision
Applicaton: Style transfer

In [ ]:
# Load a pre-trained style transfer model from TensorFlow Hub
style_transfer_model = hub.load("https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2")

In [ ]:
# Content and style image URLs
content_image_url = "https://thumbs.dreamstime.com/b/beautiful-happy-reddish-havanese-puppy-dog-sitting-frontal-looking-camera-isolated-white-background-46868560.jpg"
style_image_url = "https://sanctuarymentalhealth.org/wp-content/uploads/2021/03/The-Starry-Night-1200x630-1-979x514.jpg"

In [ ]:
# Display and load content and style images
content_img, content_array = display_image(content_image_url)
style_img, style_array = display_image(style_image_url)

In [ ]:
# Preprocess images for the model
content_image = tf.convert_to_tensor(content_array, dtype=tf.float32)
content_image = tf.image.resize(content_image, (256, 256)) / 255.0
content_image = tf.expand_dims(content_image, axis=0)

style_image = tf.convert_to_tensor(style_array, dtype=tf.float32)
style_image = tf.image.resize(style_image, (256, 256)) / 255.0
style_image = tf.expand_dims(style_image, axis=0)


In [ ]:
# Perform style transfer
stylized_image = style_transfer_model(tf.constant(content_image), tf.constant(style_image))[0]

In [ ]:
# Display the stylized image
plt.imshow(stylized_image[0])
plt.title('Stylized Image')
plt.axis('off')
plt.show()

### Task 3: How good do you think this model is? Try different images and styles and see how well it works. Can you find another pre-trained model that might work better?

More information on the model: https://www.tensorflow.org/tutorials/generative/style_transfer \
Alternative model: https://pytorch.org/tutorials/advanced/neural_style_tutorial.html

## Section 4: Language Processing
Applicaton: Similarity detection

In [ ]:
# Initialize the BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [ ]:
def get_sentence_embedding(sentence):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True, max_length=512)
    # Get the hidden states from the model
    with torch.no_grad():
        outputs = model(**inputs)
    # Take the mean of the token embeddings from the last hidden state
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.numpy()

In [ ]:
def compute_similarity(sentence1, sentence2):
    # Get sentence embeddings for both sentences
    embedding1 = get_sentence_embedding(sentence1)
    embedding2 = get_sentence_embedding(sentence2)
    # Compute cosine similarity between the embeddings
    similarity = cosine_similarity(embedding1, embedding2)[0][0]
    return similarity

In [ ]:
# Example text data
sentence1 = "AI is transforming the world."
sentence2 = "Artificial intelligence is changing everything."

In [ ]:
similarity_score = compute_similarity(sentence1, sentence2)
print(f"Similarity score: {similarity_score}")

### Task 4: Can you find the limitations of this model? Do you agree with the similarity scores? Try different sentences and see if there are any you disagree with.

### Who can get the highest similarity score?

## Extras

If you're not already familiar with it, try out DALL-E image generation \
https://openai.com/index/dall-e-3/

And ChatGPT \
https://chatgpt.com/ \
A fun exercise is to ask questions, or to complete tasks, within different contexts, e.g. 'you are a student', 'you are a teacher', 'you are a professional', etc, and see how the responses change.